# Manifold Learning for Speech Emotion Recognition
## Efthymios Tzinis

In [1]:
# Load the appropriate modules 
import os, sys, glob
import numpy as np
import cPickle as pickle
sys.path.append('../')
import config
sys.path.append(config.BASE_PATH)
from dataloader import fused_features_IEMOCAP as IEMOCAP_loader

sys.path.append(config.PATTERN_SEARCH_MDS_PATH)

In [2]:
# Session Folds Generator
def get_dataset_in_one_array(features_dic,
                             included_sessions=['Ses01', 'Ses02']):
    speaker_indices = {}
    x_all_list = []
    Y_all = []
    prev_ind = 0
    for te_speaker, te_data in features_dic.items():  
        ses_name = te_speaker[:-1]
        if not ses_name in included_sessions:
            continue
        x_all_list.append(te_data['x'])
        Y_all += te_data['y']
        this_speaker_samples = len(te_data['y'])
        
        speaker_indices[te_speaker] = (prev_ind, prev_ind + this_speaker_samples)
        prev_ind += this_speaker_samples
        X_all = np.concatenate(x_all_list, axis=0)
    return X_all, Y_all, speaker_indices, len(included_sessions)


def generate_session_folds(X_all, Y_all, features_dic, speaker_indices):
    sorted_speakers = sorted(speaker_indices)
    for i in np.arange(0, len(sorted_speakers), 2):
        sp1 = sorted_speakers[i]
        sp2 = sorted_speakers[i+1]
        
        session_name = sp1[:-1]
        
        st1, et1 = speaker_indices[sp1]
        st2, et2 = speaker_indices[sp2]
        
        Y_te = features_dic[sp1]['y'] + features_dic[sp2]['y']
        X_te = np.concatenate([X_all[st1:et1, :], X_all[st2:et2, :]], axis=0)
        
        x_tr_list = []
        Y_tr = []
        for sp in sorted_speakers:
            if sp == sp1 or sp == sp2:
                continue
            st, et = speaker_indices[sp] 
            x_tr_list.append(X_all[st:et, :])
            Y_tr += features_dic[sp]['y']
            
        X_tr = np.concatenate(x_tr_list, axis=0)    
        
        yield session_name, X_te, Y_te, X_tr, Y_tr 

def generate_folds(features_dic,
                   group_by = 'speaker'):
    if group_by == 'speaker':
        for te_speaker, te_data in features_dic.items():
            x_tr_list = []
            Y_tr = []
            for tr_speaker, tr_data in features_dic.items():
                if tr_speaker == te_speaker:
                    continue
                x_tr_list.append(tr_data['x'])
                Y_tr += tr_data['y']

            X_tr = np.concatenate(x_tr_list, axis=0)
            yield te_speaker, te_data['x'], te_data['y'], X_tr, Y_tr
     
    elif group_by == 'session':
        already_tested = []
        for te_speaker, te_data in features_dic.items():
            if not (te_speaker[:-1] in already_tested) :
                already_tested.append(te_speaker[:-1])
            else:
                continue
            X_val =  te_data['x']
            Y_val = te_data['y']
            x_tr_list = []
            Y_tr = []
            ses_name = te_speaker[:-1]
            for tr_speaker, tr_data in features_dic.items():
                if tr_speaker == te_speaker:
                    continue
                if tr_speaker[:-1] == ses_name:
                    val_speaker = tr_speaker
                    X_val = tr_data['x']
                    Y_val = tr_data['y']
                    continue
                x_tr_list.append(tr_data['x'])
                Y_tr += tr_data['y']

            X_tr = np.concatenate(x_tr_list, axis=0)
            X_ses = np.concatenate([te_data['x'], X_val], axis=0)
            Y_ses = te_data['y'] + Y_val
            yield ses_name, X_ses, Y_ses, X_tr, Y_tr
            
def fuse_excited_happiness(l):
    return ['happy + excited' 
            if (e == 'excited' or e == 'happy') 
            else e for e in l ]


In [3]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable



class IEMOCAPData(Dataset):
    def __init__(self, X,):
        self.X_high = X
    
    def __len__(self):
        return self.X_high.shape[0]
    
    def __getitem__(self, idx):
        return self.X_high[idx]

class autoencoder(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, hidden_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, in_dim)
        )

    def forward(self, x):
        h = self.encoder(x)
        x = self.decoder(h)
        return x, h


class AE(object):
    def __init__(self, original_dim, target_dim, batch_size=32, learning_rate=1e-4, num_epochs=10000, early_stop=10):
        self.original_dim = original_dim
        self.target_dim = target_dim
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.early_stop = early_stop
        self.dvc = 'cuda'
        self.model = None
    
    def fit(self, X):
        iemo_data = IEMOCAPData(X)

        dataloader = DataLoader(iemo_data, batch_size=self.batch_size, shuffle=True)
        model = autoencoder(self.original_dim, self.target_dim).to(self.dvc)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters())
        patience = 0
        prev_avg_loss = np.Inf
        for epoch in range(self.num_epochs):
            avg_loss, i = 0, 0
            for data in dataloader:
                data = torch.Tensor(data.type(torch.FloatTensor)).to(self.dvc)
                # ===================forward=====================
                output, hidden = model(data)
        #         loss = criterion(output, data, hidden)
                loss = criterion(output, data)
                # ===================backward====================
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss
                i += 1
            # ===================log========================
            if avg_loss.data[0] > prev_avg_loss:
                patience += 1
            print('epoch [{}/{}], loss:{}, patience: {}'
                  .format(epoch + 1, self.num_epochs, avg_loss.data[0] / i, patience))
            if patience >= self.early_stop:
                break
            prev_avg_loss = avg_loss.data[0]
        self.model = model
        return model
        
    def transform(self, X):
        return self.model(X)
    
    def fit_transform(self, X):
        self.model = self.fit(X)
        return self.transorm(X)

In [4]:
# Initialize all avaiulable Manifold Methods
import multidimensional
import multidimensional.common
import multidimensional.mds 
import multidimensional.smacof
import multidimensional.mds_utils as mds_utils
from sklearn import manifold, decomposition

class IdentityData(object):
    def __init__(self):
        pass 
    
    def fit_transform(self, x):
        return x

def get_manifold_methods(original_dim, target_dim):
    method_n_comp = 66
    radius_barrier = 1e-3
    explore_dim_percent = 1
    starting_radius =32
    max_turns = 100
    point_filter = (multidimensional.point_filters.FixedStochasticFilter(keep_percent=1, recalculate_each=10))
    radius_update = (multidimensional.radius_updates.AdaRadiusHalving(tolerance=.5*1e-3, burnout_tolerance=100000))

    mds_obj = multidimensional.mds.MDS(target_dim, point_filter, radius_update, starting_radius=starting_radius, 
                                       radius_barrier=radius_barrier,
                max_turns=max_turns, keep_history=False,
                explore_dim_percent=explore_dim_percent)

    manifold_methods = {
        'Pattern Search MDS': { 'results': {}, 'object': multidimensional.mds.MDS(target_dim, point_filter, 
                                                         radius_update, starting_radius=starting_radius, 
                                                         radius_barrier=radius_barrier, max_turns=max_turns, 
                                                         keep_history=False,
                                                         dissimilarities='precomputed',
                                                         explore_dim_percent=explore_dim_percent)},
        'MDS SMACOF': { 'results': {}, 'object': multidimensional.smacof.MDS(n_components=target_dim, n_init=1, 
                                                 max_iter=max_turns, dissimilarity='euclidean', n_jobs=8)},
        'LTSA': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='ltsa',n_jobs=8)},
        'Modified LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='modified',n_jobs=8)},
        'Hessian LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='hessian',n_jobs=8)},
        'LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='standard',n_jobs=8)},
        'Truncated SVD': { 'results': {}, 'object': decomposition.TruncatedSVD(n_components=target_dim)},
        'Spectral Embedding': { 'results': {}, 'object': manifold.SpectralEmbedding(n_components=target_dim, 
                                                                                    n_jobs=8)},
        'TSNE': { 'results': {}, 'object': manifold.TSNE(n_components=target_dim)},
        'ISOMAP': { 'results': {}, 'object': manifold.Isomap(12, target_dim)},
        'Original Data': { 'results': {}, 'object': IdentityData()},
        'Autoencoder': { 'results': {}, 'object': AE(original_dim, target_dim) }

    }
    return manifold_methods

In [5]:
def run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto=None):
    save_file = os.path.join('../', 'cache', saveto)
    if saveto is not None and os.path.isfile(save_file):
        with open(save_file, 'rb') as fd:
            reduced = pickle.load(fd)
        return reduced
    X_all, Y_all, speaker_indices, number_of_sessions = get_dataset_in_one_array(data_dic,
                                                                                included_sessions=included_sessions)
    # normalize the input vectors 
    X_high = StandardScaler().fit_transform(X_all)
    
    print(X_high.shape)
    
    reduced = {}
    original_dim = X_all.shape[1]
    for target_dim in target_dims:
        reduced[target_dim] = {}
        manifold_methods = get_manifold_methods(original_dim, target_dim)
    #     methods_to_test = manifold_methods.keys()
        methods_metrics = {}
        for selected_method in methods_to_test:            
            print('Checking Method: {}'.format(selected_method))
            
            print('Reducing Input from Dimension: {} to a Lower Embedded Manifold with dimensions: {}...'.format(
                   X_high.shape[1], target_dim))
            try:
                obj = manifold_methods[selected_method]['object']
                if selected_method == 'Pattern Search MDS':
                    d_goal = multidimensional.common.DISTANCE_MATRIX(X_high.astype(np.float64))
    #                 d_goal = 1.0 - np.corrcoef(X_high.astype(np.float64))
    #                 np.fill_diagonal(d_goal, 0)
                    X_low = obj.fit_transform(d_goal)
                else:
                    X_low = obj.fit_transform(X_high)
            except Exception as e:
                print(e)
                X_low = None
            reduced[target_dim][selected_method] = X_low
    if saveto is not None:
        with open(save_file, 'wb') as fd:
            pickle.dump(reduced, fd)
    return reduced

In [6]:
# Find the best performing nonlinear features for KNN classification after dimensionality reduction
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pprint 
import pandas as pd 

def run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions):
    _, Y_all, speaker_indices, number_of_sessions = get_dataset_in_one_array(
        data_dic, included_sessions=included_sessions)
    df_results = {}
    for target_dim, methods in reduced.iteritems():
        methods_metrics = {}
        for selected_method, X_low in methods.iteritems():
            print('Checking Method: {}'.format(selected_method))
            metrics_l = {'uw_acc': dict([(k, 0.0) for k in n_neighbors]), 'w_acc': dict([(k, 0.0) for k in n_neighbors])}
            if X_low is None:
                methods_metrics[selected_method+' UA'] = metrics_l['uw_acc']
                methods_metrics[selected_method+' WA'] = metrics_l['w_acc']
                continue
                
            for k in n_neighbors:
    #             print 'Testing for Nearest Neighbors: K={}'.format(k)
                knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', algorithm='brute', leaf_size=30, 
                                           p=2, metric='minkowski', metric_params=None, n_jobs=8)

                session_folds = generate_session_folds(X_low, Y_all, data_dic, speaker_indices)
                for session, X_te, Y_te, X_tr, Y_tr in session_folds:
    #                 print "Testing for Session: {}".format(session)
                    Y_te, Y_tr = fuse_excited_happiness(Y_te), fuse_excited_happiness(Y_tr)
                    
                    try:
                        knn.fit(X_tr, Y_tr) 
                        Y_predicted = knn.predict(X_te)

                        w_acc = accuracy_score(Y_predicted, Y_te)
                        cmat = confusion_matrix(Y_te, Y_predicted)
                        with np.errstate(divide='ignore'):
                            uw_acc = (cmat.diagonal() / (1.0 * cmat.sum(axis=1) + 1e-6)).mean()
                            if np.isnan(uw_acc):
                                uw_acc = 0.
                        w_acc = round(w_acc*100,1)
                        uw_acc = round(uw_acc*100,1)
                        metrics_l['uw_acc'][k] += uw_acc/number_of_sessions
                        metrics_l['w_acc'][k] += w_acc/number_of_sessions
                    except:
                        metrics_l['uw_acc'][k] += 0.
                        metrics_l['w_acc'][k] += 0.
    #             print 'Done'
            methods_metrics[selected_method+' UA'] = metrics_l['uw_acc']
            methods_metrics[selected_method+' WA'] = metrics_l['w_acc']
#             pprint.pprint(metrics_l)

        df = pd.DataFrame.from_dict(methods_metrics, orient="index")
        df_results[target_dim] = df[sorted(df.columns)]
        
    return df_results

In [9]:
# Define parameters for IEMOCAP Session Experiments 
n_neighbors = np.arange(1, 40, 4)

# Find all appropriate files 
IEMOCAP_data_path = '/home/george/all_TRUE_IEMOCAP_feats/'
l_feats_p = IEMOCAP_data_path + 'linear/IEMOCAP_linear_emobase2010'
# nl_feats_l = glob.glob( IEMOCAP_data_path + '/utterance/*.dat')
# nl_feats_p = nl_feats_l.pop()
nl_feats_p = os.path.join(IEMOCAP_data_path, 
             'utterance/IEMOCAP-rqa-ad_hoc-tau-7-supremum-recurrence_rate-0.15-dur-0.03-fs-16000.dat')
included_sessions=['Ses01', 'Ses02', 'Ses03', 'Ses04', 'Ses05']
methods_to_test = ['Autoencoder', 'Pattern Search MDS', 'Truncated SVD', 'Spectral Embedding', 
                   'LLE', 'Modified LLE', 'ISOMAP'] 

In [8]:
target_dims = [50]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([l_feats_p, nl_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr50-fused-iemocap.p')
original_results = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)
fused_results1 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)
with open('dr50-fused-iemocap-results.p', 'wb') as fd:
    pickle.dump(fused_results1, fd)
    
with open('fused-iemocap-original-results.p', 'wb') as fd:
    pickle.dump(original_results, fd)

(5531, 2014)
Checking Method: Autoencoder
Reducing Input from Dimension: 2014 to a Lower Embedded Manifold with dimensions: 50...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:78: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:84: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/10000], loss:0.642158389091, patience: 0
epoch [2/10000], loss:0.502191781998, patience: 0
epoch [3/10000], loss:0.459500998259, patience: 0
epoch [4/10000], loss:0.43812212348, patience: 0
epoch [5/10000], loss:0.419775217772, patience: 0
epoch [6/10000], loss:0.406265258789, patience: 0
epoch [7/10000], loss:0.396254956722, patience: 0
epoch [8/10000], loss:0.388491630554, patience: 0
epoch [9/10000], loss:0.380748003721, patience: 0
epoch [10/10000], loss:0.372565329075, patience: 0
epoch [11/10000], loss:0.368515461683, patience: 0
epoch [12/10000], loss:0.365899145603, patience: 0
epoch [13/10000], loss:0.362794071436, patience: 0
epoch [14/10000], loss:0.356867492199, patience: 0
epoch [15/10000], loss:0.354566812515, patience: 0
epoch [16/10000], loss:0.347530305386, patience: 0
epoch [17/10000], loss:0.341391742229, patience: 0
epoch [18/10000], loss:0.34833586216, patience: 1
epoch [19/10000], loss:0.34206071496, patience: 1
epoch [20/10000], loss:0.337718635798, pati

INFO:MDS:Starting Error: 56293989293.6
INFO:MDS:Epoch took: 20.6644258499
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (56293989293.6, 49001607390.6, 7292381903.01)
INFO:MDS:Epoch took: 21.7581989765
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (7292381903.01, 5300749349.09, 1991632553.92)
INFO:MDS:Epoch took: 22.2926840782
INFO:MDS:Turn 3: Radius 32: (prev, error decrease, error): (1991632553.92, 256110532.903, 1735522021.01)
INFO:MDS:Epoch took: 21.6373939514
INFO:MDS:Turn 4: Radius 32: (prev, error decrease, error): (1735522021.01, 253933893.817, 1481588127.2)
INFO:MDS:Epoch took: 20.7192971706
INFO:MDS:Turn 5: Radius 32: (prev, error decrease, error): (1481588127.2, 116088840.907, 1365499286.29)
INFO:MDS:Epoch took: 21.9112110138
INFO:MDS:Turn 6: Radius 32: (prev, error decrease, error): (1365499286.29, -4972011.76016, 1370471298.05)
INFO:MDS:Epoch took: 22.213037014
INFO:MDS:Turn 7: Radius 16.0: (prev, error decrease, error): (1370471298.05, 681609195.5

INFO:MDS:Epoch took: 21.9875910282
INFO:MDS:Turn 59: Radius 4.0: (prev, error decrease, error): (160402617.852, 260638.158177, 160141979.694)
INFO:MDS:Epoch took: 20.9490330219
INFO:MDS:Turn 60: Radius 4.0: (prev, error decrease, error): (160141979.694, 263053.039591, 159878926.655)
INFO:MDS:Epoch took: 20.6846671104
INFO:MDS:Turn 61: Radius 4.0: (prev, error decrease, error): (159878926.655, 264979.39524, 159613947.259)
INFO:MDS:Epoch took: 20.7139220238
INFO:MDS:Turn 62: Radius 4.0: (prev, error decrease, error): (159613947.259, 224961.784242, 159388985.475)
INFO:MDS:Epoch took: 20.7003281116
INFO:MDS:Turn 63: Radius 4.0: (prev, error decrease, error): (159388985.475, 154469.284531, 159234516.191)
INFO:MDS:Epoch took: 20.8918869495
INFO:MDS:Turn 64: Radius 4.0: (prev, error decrease, error): (159234516.191, 178171.332769, 159056344.858)
INFO:MDS:Epoch took: 22.6670110226
INFO:MDS:Turn 65: Radius 4.0: (prev, error decrease, error): (159056344.858, 132865.970275, 158923478.888)
INFO:MD

MDS.fit_transform took: 2109.20285583 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 2014 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 2014 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: LLE
Reducing Input from Dimension: 2014 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: Modified LLE
Reducing Input from Dimension: 2014 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: ISOMAP
Reducing Input from Dimension: 2014 to a Lower Embedded Manifold with dimensions: 50...


TypeError: run_IEMOCAP_session_KNN() takes exactly 4 arguments (5 given)

In [ ]:
target_dims = [100]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([l_feats_p, nl_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr100-fused-iemocap.p')
# original_results = run_IEMOCAP_session_KNN(n_neighbors, [2014], ['Original Data'], data_dic, included_sessions)
fused_results2 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)
with open('dr100-fused-iemocap-results.p', 'wb') as fd:
    pickle.dump(fused_results2, fd)

In [ ]:
target_dims = [25, 10, 5, 3, 2]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([l_feats_p, nl_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr25-10-5-3-2-fused-iemocap.p')
# original_results = run_IEMOCAP_session_KNN(n_neighbors, [2014], ['Original Data'], data_dic, included_sessions)
fused_results3 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)
with open('dr25-10-5-3-2-fused-iemocap-results.p', 'wb') as fd:
    pickle.dump(fused_results3, fd)

In [ ]:
from IPython.display import display
for target_dim in sorted(original_results.keys()):
    df = original_results[target_dim]
    print "Using Original Data"
    print display(df)

for target_dim in sorted(fused_results1.keys()):
    df = fused_results1[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    
for target_dim in sorted(fused_results2.keys()):
    df = fused_results2[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    
for target_dim in sorted(fused_results3.keys()):
    df = fused_results3[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)

In [10]:
target_dims = [50]
data_dic = IEMOCAP_loader.get_fused_features([nl_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr50-rqa-iemocap.p')
original_results = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)
nl_results1 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)

with open('dr50-rqa-iemocap-results.p', 'wb') as fd:
    pickle.dump(nl_results1, fd)
    
with open('rqa-original-iemocap-results.p', 'wb') as fd:
    pickle.dump(original_results, fd)


Checking Method: Pattern Search MDS
Checking Method: Autoencoder
Checking Method: Spectral Embedding
Checking Method: Modified LLE
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE
Checking Method: Pattern Search MDS
Checking Method: Autoencoder
Checking Method: Spectral Embedding
Checking Method: Modified LLE
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE
Checking Method: Pattern Search MDS
Checking Method: Autoencoder
Checking Method: Spectral Embedding
Checking Method: Modified LLE
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE
Checking Method: Pattern Search MDS
Checking Method: Autoencoder
Checking Method: Spectral Embedding
Checking Method: Modified LLE
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE
Checking Method: Pattern Search MDS
Checking Method: Autoencoder
Checking Method: Spectral Embedding
Checking Method: Modified LLE
Checking Method: Truncated SVD
Checking M

In [11]:
target_dims = [100]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([nl_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr100-rqa-iemocap.p')
# original_results = run_IEMOCAP_session_KNN(n_neighbors, [2014], ['Original Data'], data_dic, included_sessions)
nl_results2 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)

with open('dr100-rqa-iemocap-results.p', 'wb') as fd:
    pickle.dump(nl_results2, fd)

(5531, 432)
Checking Method: Autoencoder
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 100...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:78: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:84: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/10000], loss:0.491886466742, patience: 0
epoch [2/10000], loss:0.305434912443, patience: 0
epoch [3/10000], loss:0.262733638287, patience: 0
epoch [4/10000], loss:0.233502924442, patience: 0
epoch [5/10000], loss:0.213060304523, patience: 0
epoch [6/10000], loss:0.202687293291, patience: 0
epoch [7/10000], loss:0.187015548348, patience: 0
epoch [8/10000], loss:0.182442292571, patience: 0
epoch [9/10000], loss:0.16657435894, patience: 0
epoch [10/10000], loss:0.163442999125, patience: 0
epoch [11/10000], loss:0.152704775333, patience: 0
epoch [12/10000], loss:0.149173751473, patience: 0
epoch [13/10000], loss:0.141414508224, patience: 0
epoch [14/10000], loss:0.139071539044, patience: 0
epoch [15/10000], loss:0.133042767644, patience: 0
epoch [16/10000], loss:0.131865620613, patience: 0
epoch [17/10000], loss:0.126175716519, patience: 0
epoch [18/10000], loss:0.119496636093, patience: 0
epoch [19/10000], loss:0.12067233026, patience: 1
epoch [20/10000], loss:0.119143642485, pat

INFO:MDS:Starting Error: 9951505535.4
INFO:MDS:Epoch took: 40.4384520054
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (9951505535.4, 5257129091.21, 4694376444.19)
INFO:MDS:Epoch took: 40.0997920036
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (4694376444.19, -440916075.108, 5135292519.29)
INFO:MDS:Epoch took: 40.0845990181
INFO:MDS:Turn 3: Radius 16.0: (prev, error decrease, error): (5135292519.29, 3889160551.55, 1246131967.75)
INFO:MDS:Epoch took: 40.8200359344
INFO:MDS:Turn 4: Radius 16.0: (prev, error decrease, error): (1246131967.75, 696519293.521, 549612674.227)
INFO:MDS:Epoch took: 39.9056220055
INFO:MDS:Turn 5: Radius 16.0: (prev, error decrease, error): (549612674.227, 64841416.7803, 484771257.447)
INFO:MDS:Epoch took: 39.989248991
INFO:MDS:Turn 6: Radius 16.0: (prev, error decrease, error): (484771257.447, 104458365.485, 380312891.962)
INFO:MDS:Epoch took: 39.8059060574
INFO:MDS:Turn 7: Radius 16.0: (prev, error decrease, error): (380312891.962, -48

INFO:MDS:Epoch took: 40.160410881
INFO:MDS:Turn 59: Radius 2.0: (prev, error decrease, error): (22206886.9415, 181618.740782, 22025268.2007)
INFO:MDS:Epoch took: 40.4386198521
INFO:MDS:Turn 60: Radius 2.0: (prev, error decrease, error): (22025268.2007, 145240.795658, 21880027.4051)
INFO:MDS:Epoch took: 40.1314001083
INFO:MDS:Turn 61: Radius 2.0: (prev, error decrease, error): (21880027.4051, 131998.971766, 21748028.4333)
INFO:MDS:Epoch took: 40.0578691959
INFO:MDS:Turn 62: Radius 2.0: (prev, error decrease, error): (21748028.4333, 121741.699553, 21626286.7338)
INFO:MDS:Epoch took: 40.3745160103
INFO:MDS:Turn 63: Radius 2.0: (prev, error decrease, error): (21626286.7338, 97547.7539429, 21528738.9798)
INFO:MDS:Epoch took: 40.2254760265
INFO:MDS:Turn 64: Radius 2.0: (prev, error decrease, error): (21528738.9798, 86078.2159194, 21442660.7639)
INFO:MDS:Epoch took: 40.2647988796
INFO:MDS:Turn 65: Radius 2.0: (prev, error decrease, error): (21442660.7639, 68510.9410965, 21374149.8228)
INFO:MD

MDS.fit_transform took: 4010.94608808 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: LLE
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: Modified LLE
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 100...
modified LLE requires n_neighbors >= n_components
Checking Method: ISOMAP
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE


In [12]:
target_dims = [25, 10, 5, 3, 2]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([nl_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr25-10-5-3-2-rqa-iemocap.p')
# original_results = run_IEMOCAP_session_KNN(n_neighbors, [2014], ['Original Data'], data_dic, included_sessions)
nl_results3 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)

with open('dr25-10-5-3-2-rqa-iemocap-results.p', 'wb') as fd:
    pickle.dump(nl_results3, fd)

(5531, 432)
Checking Method: Autoencoder
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 25...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:78: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:84: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/10000], loss:0.516465008259, patience: 0
epoch [2/10000], loss:0.318490356207, patience: 0
epoch [3/10000], loss:0.271252065897, patience: 0
epoch [4/10000], loss:0.248410761356, patience: 0
epoch [5/10000], loss:0.22765044868, patience: 0
epoch [6/10000], loss:0.215372264385, patience: 0
epoch [7/10000], loss:0.204233258963, patience: 0
epoch [8/10000], loss:0.196878060699, patience: 0
epoch [9/10000], loss:0.187517747283, patience: 0
epoch [10/10000], loss:0.187059655786, patience: 0
epoch [11/10000], loss:0.18127001822, patience: 0
epoch [12/10000], loss:0.177678093314, patience: 0
epoch [13/10000], loss:0.174681305885, patience: 0
epoch [14/10000], loss:0.165179863572, patience: 0
epoch [15/10000], loss:0.169323891401, patience: 1
epoch [16/10000], loss:0.163904234767, patience: 1
epoch [17/10000], loss:0.157292276621, patience: 1
epoch [18/10000], loss:0.158239409328, patience: 2
epoch [19/10000], loss:0.156746640801, patience: 2
epoch [20/10000], loss:0.155647724867, pat

INFO:MDS:Starting Error: 11529460998.4
INFO:MDS:Epoch took: 10.1837010384
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (11529460998.4, 6661951823.46, 4867509174.98)
INFO:MDS:Epoch took: 10.5306038857
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (4867509174.98, -888927524.734, 5756436699.72)
INFO:MDS:Epoch took: 10.2754440308
INFO:MDS:Turn 3: Radius 16.0: (prev, error decrease, error): (5756436699.72, 4301910634.02, 1454526065.69)
INFO:MDS:Epoch took: 10.2081699371
INFO:MDS:Turn 4: Radius 16.0: (prev, error decrease, error): (1454526065.69, 828758475.976, 625767589.715)
INFO:MDS:Epoch took: 10.4718370438
INFO:MDS:Turn 5: Radius 16.0: (prev, error decrease, error): (625767589.715, 112857067.497, 512910522.218)
INFO:MDS:Epoch took: 10.273914814
INFO:MDS:Turn 6: Radius 16.0: (prev, error decrease, error): (512910522.218, 96862366.3322, 416048155.886)
INFO:MDS:Epoch took: 10.3972239494
INFO:MDS:Turn 7: Radius 16.0: (prev, error decrease, error): (416048155.886, -

INFO:MDS:Epoch took: 10.1794281006
INFO:MDS:Turn 59: Radius 2.0: (prev, error decrease, error): (35001409.4287, 913338.599886, 34088070.8289)
INFO:MDS:Epoch took: 10.0276129246
INFO:MDS:Turn 60: Radius 2.0: (prev, error decrease, error): (34088070.8289, 761580.169769, 33326490.6591)
INFO:MDS:Epoch took: 10.1175279617
INFO:MDS:Turn 61: Radius 2.0: (prev, error decrease, error): (33326490.6591, 646381.372882, 32680109.2862)
INFO:MDS:Epoch took: 10.2125220299
INFO:MDS:Turn 62: Radius 2.0: (prev, error decrease, error): (32680109.2862, 586595.461117, 32093513.8251)
INFO:MDS:Epoch took: 10.0247769356
INFO:MDS:Turn 63: Radius 2.0: (prev, error decrease, error): (32093513.8251, 502043.135742, 31591470.6893)
INFO:MDS:Epoch took: 10.0367941856
INFO:MDS:Turn 64: Radius 2.0: (prev, error decrease, error): (31591470.6893, 415138.598823, 31176332.0905)
INFO:MDS:Epoch took: 10.210119009
INFO:MDS:Turn 65: Radius 2.0: (prev, error decrease, error): (31176332.0905, 373382.115521, 30802949.975)
INFO:MDS

...

INFO:MDS:Starting Error: 12653931517.7
INFO:MDS:Epoch took: 1.65726113319
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (12653931517.7, 6942235782.17, 5711695735.52)
INFO:MDS:Epoch took: 1.57681393623
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (5711695735.52, -391956435.74, 6103652171.26)
INFO:MDS:Epoch took: 1.70116591454
INFO:MDS:Turn 3: Radius 16.0: (prev, error decrease, error): (6103652171.26, 4192921918.66, 1910730252.61)
INFO:MDS:Epoch took: 1.52877902985
INFO:MDS:Turn 4: Radius 16.0: (prev, error decrease, error): (1910730252.61, 493361538.997, 1417368713.61)
INFO:MDS:Epoch took: 1.5032761097
INFO:MDS:Turn 5: Radius 16.0: (prev, error decrease, error): (1417368713.61, 116688676.039, 1300680037.57)
INFO:MDS:Epoch took: 1.47897410393
INFO:MDS:Turn 6: Radius 16.0: (prev, error decrease, error): (1300680037.57, 45793809.7393, 1254886227.83)
INFO:MDS:Epoch took: 1.47004699707
INFO:MDS:Turn 7: Radius 16.0: (prev, error decrease, error): (1254886227.83, 15

INFO:MDS:Epoch took: 1.43970918655
INFO:MDS:Turn 59: Radius 0.25: (prev, error decrease, error): (451316708.605, 101169.962443, 451215538.642)
INFO:MDS:Epoch took: 1.44012784958
INFO:MDS:Turn 60: Radius 0.125: (prev, error decrease, error): (451215538.642, 116782.688318, 451098755.954)
INFO:MDS:Epoch took: 1.45418286324
INFO:MDS:Turn 61: Radius 0.0625: (prev, error decrease, error): (451098755.954, 41954.1500221, 451056801.804)
INFO:MDS:Epoch took: 1.51174902916
INFO:MDS:Turn 62: Radius 0.03125: (prev, error decrease, error): (451056801.804, 16988.740647, 451039813.063)
INFO:MDS:Epoch took: 1.47319316864
INFO:MDS:Turn 63: Radius 0.015625: (prev, error decrease, error): (451039813.063, 7262.49052316, 451032550.573)
INFO:MDS:Epoch took: 1.46095395088
INFO:MDS:Turn 64: Radius 0.0078125: (prev, error decrease, error): (451032550.573, 3262.6802122, 451029287.893)
INFO:MDS:Epoch took: 1.4499771595
INFO:MDS:Turn 65: Radius 0.00390625: (prev, error decrease, error): (451029287.893, 1533.259422

MDS.fit_transform took: 101.918693066 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: LLE
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: Modified LLE
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: ISOMAP
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: Autoencoder
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 2...
epoch [1/10000], loss:0.621658086777, patience: 0
epoch [2/10000], loss:0.565931975842, patience: 0
epoch [3/10000], loss:0.55063867569, patience: 0
epoch [4/10000], loss:0.543417036533, patience: 0
epoch [5/10000], loss:0.538463830948, patience: 0
epoch [6/10000], loss:

INFO:MDS:Starting Error: 12771694098.6
INFO:MDS:Epoch took: 1.05404901505
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (12771694098.6, 6086811670.21, 6684882428.41)
INFO:MDS:Epoch took: 1.04979491234
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (6684882428.41, -306909897.967, 6991792326.38)
INFO:MDS:Epoch took: 1.05123090744
INFO:MDS:Turn 3: Radius 16.0: (prev, error decrease, error): (6991792326.38, 4464817744.22, 2526974582.16)
INFO:MDS:Epoch took: 1.05291414261
INFO:MDS:Turn 4: Radius 16.0: (prev, error decrease, error): (2526974582.16, 198917683.789, 2328056898.37)
INFO:MDS:Epoch took: 1.13560700417
INFO:MDS:Turn 5: Radius 16.0: (prev, error decrease, error): (2328056898.37, 197728478.931, 2130328419.44)
INFO:MDS:Epoch took: 1.24779987335
INFO:MDS:Turn 6: Radius 16.0: (prev, error decrease, error): (2130328419.44, -30818587.9027, 2161147007.34)
INFO:MDS:Epoch took: 1.05872607231
INFO:MDS:Turn 7: Radius 8.0: (prev, error decrease, error): (2161147007.34, 

MDS.fit_transform took: 45.9373059273 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: LLE
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: Modified LLE
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: ISOMAP
Reducing Input from Dimension: 432 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Method: 

In [13]:
from IPython.display import display
for target_dim in sorted(original_results.keys()):
    df = original_results[target_dim]
    print "Using Original Data"
    print display(df)

for target_dim in sorted(nl_results1.keys()):
    df = nl_results1[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    
for target_dim in sorted(nl_results2.keys()):
    df = nl_results2[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    
for target_dim in sorted(nl_results3.keys()):
    df = nl_results3[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)

Using Original Data


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,35.40,37.60,39.08,40.20,39.60,39.70,39.92,40.84,41.10,40.74
ISOMAP WA,34.90,37.38,38.76,39.94,39.32,39.54,39.68,40.58,40.84,40.34
LLE UA,34.28,36.76,39.12,40.56,41.34,41.68,41.12,41.36,41.56,41.92
LLE WA,33.76,36.42,38.64,40.10,41.06,41.34,40.96,41.30,41.38,41.86
Modified LLE UA,27.60,28.38,29.98,29.60,29.82,29.94,30.04,30.52,30.30,30.94
Modified LLE WA,28.12,29.08,30.72,30.68,30.70,30.82,30.90,31.34,31.08,31.72
Pattern Search MDS UA,34.08,36.70,37.80,39.04,39.38,40.00,40.06,40.66,41.02,40.10
Pattern Search MDS WA,34.20,36.80,37.88,39.38,39.60,40.30,40.30,40.72,41.06,40.12


None
Using Original Data


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,38.18,39.44,41.06,41.38,42.42,41.90,43.10,43.70,43.98,44.36
ISOMAP WA,38.10,39.14,40.82,41.48,42.46,42.06,43.26,43.52,43.74,44.10
LLE UA,34.30,37.60,39.38,40.28,41.24,41.66,42.00,41.46,42.46,42.66
LLE WA,33.92,37.28,38.90,39.84,40.88,41.44,41.84,41.34,42.26,42.46
Modified LLE UA,35.60,38.88,41.28,41.72,42.40,42.66,42.78,42.80,42.70,42.66
Modified LLE WA,35.34,38.26,40.40,40.94,41.74,41.94,41.92,41.94,41.82,41.84
Pattern Search MDS UA,35.74,39.44,40.88,42.04,43.12,43.46,43.34,43.92,44.08,44.16
Pattern Search MDS WA,35.44,39.14,40.56,41.76,42.84,43.36,43.10,43.88,43.94,44.02


None
Using Original Data


...

None
For Target Dimension: 10


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,39.98,42.14,43.80,44.56,44.76,45.48,46.36,46.64,47.44,47.52
ISOMAP WA,39.26,41.24,42.42,43.52,43.56,44.34,45.04,45.32,46.10,46.14
LLE UA,38.70,41.92,43.60,45.12,45.26,45.08,45.54,46.06,46.04,45.84
LLE WA,38.38,41.12,43.12,44.78,45.00,44.92,45.14,45.74,45.90,45.68
Modified LLE UA,38.48,40.94,42.56,43.44,43.82,44.48,45.26,45.42,45.64,46.16
Modified LLE WA,38.10,40.28,42.08,42.76,43.10,43.92,44.78,44.82,45.08,45.60
Pattern Search MDS UA,39.12,42.24,44.46,45.16,45.84,46.42,46.74,47.14,46.98,47.10
Pattern Search MDS WA,38.44,41.56,43.12,43.82,44.62,44.98,45.36,45.68,45.58,45.64


None
For Target Dimension: 25


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,40.18,42.38,44.98,45.84,46.34,46.38,46.76,47.14,46.98,47.24
ISOMAP WA,39.20,41.20,43.82,44.60,44.74,44.88,45.22,45.60,45.50,45.76
LLE UA,39.96,42.50,44.50,45.84,46.40,46.82,46.88,47.36,47.84,47.52
LLE WA,39.68,42.42,43.98,45.30,46.10,46.46,46.40,46.98,47.58,47.18
Modified LLE UA,41.16,43.46,44.80,46.12,46.92,47.82,48.34,48.36,48.54,48.68
Modified LLE WA,40.74,42.64,44.10,45.32,46.10,47.10,47.48,47.42,47.62,47.94
Pattern Search MDS UA,38.88,43.12,45.12,46.58,47.06,47.70,47.98,48.52,48.44,48.22
Pattern Search MDS WA,38.22,42.18,43.86,45.32,45.78,46.28,46.64,47.30,47.02,46.80


None


In [14]:
target_dims = [50]
data_dic = IEMOCAP_loader.get_fused_features([l_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr50-emobase-iemocap.p')
original_results = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)
emobase_results1 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)

with open('dr50-emobase-iemocap-results.p', 'wb') as fd:
    pickle.dump(emobase_results1, fd)
    
with open('emobase-original-iemocap-results.p', 'wb') as fd:
    pickle.dump(original_results, fd)

(5531, 1582)
Checking Method: Autoencoder
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 50...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:78: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:84: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/10000], loss:0.656702399254, patience: 0
epoch [2/10000], loss:0.523653686047, patience: 0
epoch [3/10000], loss:0.485359579325, patience: 0
epoch [4/10000], loss:0.458398669958, patience: 0
epoch [5/10000], loss:0.439420998096, patience: 0
epoch [6/10000], loss:0.426185071468, patience: 0
epoch [7/10000], loss:0.41131311655, patience: 0
epoch [8/10000], loss:0.402097225189, patience: 0
epoch [9/10000], loss:0.393391549587, patience: 0
epoch [10/10000], loss:0.388658821583, patience: 0
epoch [11/10000], loss:0.37865498662, patience: 0
epoch [12/10000], loss:0.374077111483, patience: 0
epoch [13/10000], loss:0.370232224464, patience: 0
epoch [14/10000], loss:0.367345541716, patience: 0
epoch [15/10000], loss:0.363628298044, patience: 0
epoch [16/10000], loss:0.355877041817, patience: 0
epoch [17/10000], loss:0.351962864399, patience: 0
epoch [18/10000], loss:0.349390953779, patience: 0
epoch [19/10000], loss:0.347624480724, patience: 0
epoch [20/10000], loss:0.344209045172, pat

INFO:MDS:Starting Error: 43715934135.3
INFO:MDS:Epoch took: 20.2480449677
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (43715934135.3, 39684440587.4, 4031493547.81)
INFO:MDS:Epoch took: 20.3016388416
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (4031493547.81, 2202381955.36, 1829111592.45)
INFO:MDS:Epoch took: 20.2367081642
INFO:MDS:Turn 3: Radius 32: (prev, error decrease, error): (1829111592.45, 226375801.823, 1602735790.63)
INFO:MDS:Epoch took: 20.2179789543
INFO:MDS:Turn 4: Radius 32: (prev, error decrease, error): (1602735790.63, 94365003.608, 1508370787.02)
INFO:MDS:Epoch took: 20.3146700859
INFO:MDS:Turn 5: Radius 32: (prev, error decrease, error): (1508370787.02, 42805743.6765, 1465565043.35)
INFO:MDS:Epoch took: 19.8559479713
INFO:MDS:Turn 6: Radius 32: (prev, error decrease, error): (1465565043.35, 49068323.6283, 1416496719.72)
INFO:MDS:Epoch took: 20.2240068913
INFO:MDS:Turn 7: Radius 32: (prev, error decrease, error): (1416496719.72, -2625067.128

INFO:MDS:Epoch took: 19.9890320301
INFO:MDS:Turn 59: Radius 4.0: (prev, error decrease, error): (132827763.256, 167410.892694, 132660352.363)
INFO:MDS:Epoch took: 19.8304169178
INFO:MDS:Turn 60: Radius 4.0: (prev, error decrease, error): (132660352.363, 146044.7543, 132514307.609)
INFO:MDS:Epoch took: 19.8497960567
INFO:MDS:Turn 61: Radius 4.0: (prev, error decrease, error): (132514307.609, 94288.208534, 132420019.4)
INFO:MDS:Epoch took: 19.9736301899
INFO:MDS:Turn 62: Radius 4.0: (prev, error decrease, error): (132420019.4, 78076.6119107, 132341942.788)
INFO:MDS:Epoch took: 19.9424290657
INFO:MDS:Turn 63: Radius 4.0: (prev, error decrease, error): (132341942.788, 102924.33589, 132239018.453)
INFO:MDS:Epoch took: 19.8263270855
INFO:MDS:Turn 64: Radius 4.0: (prev, error decrease, error): (132239018.453, 66802.6507723, 132172215.802)
INFO:MDS:Epoch took: 20.0505669117
INFO:MDS:Turn 65: Radius 4.0: (prev, error decrease, error): (132172215.802, 64975.2162329, 132107240.586)
INFO:MDS:Epoch

MDS.fit_transform took: 1996.776052 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: Modified LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: ISOMAP
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 50...
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking 

In [15]:
target_dims = [100]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([l_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr100-emobase-iemocap.p')
# original_results = run_IEMOCAP_session_KNN(n_neighbors, [2014], ['Original Data'], data_dic, included_sessions)
emobase_results2 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)

with open('dr100-emobase-iemocap-results.p', 'wb') as fd:
    pickle.dump(emobase_results2, fd)

(5531, 1582)
Checking Method: Autoencoder
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 100...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:78: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:84: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/10000], loss:0.653323352337, patience: 0
epoch [2/10000], loss:0.523835837841, patience: 0
epoch [3/10000], loss:0.478292673826, patience: 0
epoch [4/10000], loss:0.450199395418, patience: 0
epoch [5/10000], loss:0.432233929634, patience: 0
epoch [6/10000], loss:0.41868558526, patience: 0
epoch [7/10000], loss:0.405824273825, patience: 0
epoch [8/10000], loss:0.395403057337, patience: 0
epoch [9/10000], loss:0.385528862476, patience: 0
epoch [10/10000], loss:0.382547497749, patience: 0
epoch [11/10000], loss:0.371728777885, patience: 0
epoch [12/10000], loss:0.371584683657, patience: 0
epoch [13/10000], loss:0.359058976173, patience: 0
epoch [14/10000], loss:0.362813621759, patience: 1
epoch [15/10000], loss:0.355430930853, patience: 1
epoch [16/10000], loss:0.350970715284, patience: 1
epoch [17/10000], loss:0.343931019306, patience: 1
epoch [18/10000], loss:0.340179681778, patience: 1
epoch [19/10000], loss:0.338780015707, patience: 1
epoch [20/10000], loss:0.333917081356, pa

INFO:MDS:Starting Error: 41820243958.9
INFO:MDS:Epoch took: 40.2752590179
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (41820243958.9, 37969016311.1, 3851227647.81)
INFO:MDS:Epoch took: 40.2168638706
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (3851227647.81, 2083357395.7, 1767870252.11)
INFO:MDS:Epoch took: 40.418555975
INFO:MDS:Turn 3: Radius 32: (prev, error decrease, error): (1767870252.11, 282801969.581, 1485068282.53)
INFO:MDS:Epoch took: 39.9547529221
INFO:MDS:Turn 4: Radius 32: (prev, error decrease, error): (1485068282.53, 113065620.785, 1372002661.75)
INFO:MDS:Epoch took: 40.1916408539
INFO:MDS:Turn 5: Radius 32: (prev, error decrease, error): (1372002661.75, 40976196.1579, 1331026465.59)
INFO:MDS:Epoch took: 40.0492479801
INFO:MDS:Turn 6: Radius 32: (prev, error decrease, error): (1331026465.59, 12011978.3683, 1319014487.22)
INFO:MDS:Epoch took: 40.2463710308
INFO:MDS:Turn 7: Radius 32: (prev, error decrease, error): (1319014487.22, 10787445.6245

INFO:MDS:Epoch took: 40.2224628925
INFO:MDS:Turn 59: Radius 2.0: (prev, error decrease, error): (86375138.5148, 3082512.96214, 83292625.5527)
INFO:MDS:Epoch took: 40.6938271523
INFO:MDS:Turn 60: Radius 2.0: (prev, error decrease, error): (83292625.5527, 2103211.97753, 81189413.5751)
INFO:MDS:Epoch took: 40.2683382034
INFO:MDS:Turn 61: Radius 2.0: (prev, error decrease, error): (81189413.5751, 1638924.54366, 79550489.0315)
INFO:MDS:Epoch took: 40.3206660748
INFO:MDS:Turn 62: Radius 2.0: (prev, error decrease, error): (79550489.0315, 1388453.67127, 78162035.3602)
INFO:MDS:Epoch took: 40.6795399189
INFO:MDS:Turn 63: Radius 2.0: (prev, error decrease, error): (78162035.3602, 1195230.69692, 76966804.6633)
INFO:MDS:Epoch took: 40.3215780258
INFO:MDS:Turn 64: Radius 2.0: (prev, error decrease, error): (76966804.6633, 1070080.71272, 75896723.9506)
INFO:MDS:Epoch took: 40.1523339748
INFO:MDS:Turn 65: Radius 2.0: (prev, error decrease, error): (75896723.9506, 959340.359536, 74937383.591)
INFO:MD

MDS.fit_transform took: 4328.43480897 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: Modified LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 100...
modified LLE requires n_neighbors >= n_components
Checking Method: ISOMAP
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 100...
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE


In [16]:
target_dims = [25, 10, 5, 3, 2]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([l_feats_p])
reduced = run_DR(target_dims, methods_to_test, data_dic, included_sessions, saveto='dr25-10-5-3-2-emobase-iemocap.p')
# original_results = run_IEMOCAP_session_KNN(n_neighbors, [2014], ['Original Data'], data_dic, included_sessions)
emobase_results3 = run_IEMOCAP_session_KNN(n_neighbors, reduced, data_dic, included_sessions)

with open('dr25-10-5-3-2-emobase-iemocap-results.p', 'wb') as fd:
    pickle.dump(emobase_results3, fd)

(5531, 1582)
Checking Method: Autoencoder
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 25...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:78: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:84: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/10000], loss:0.656466126442, patience: 0
epoch [2/10000], loss:0.524454116821, patience: 0
epoch [3/10000], loss:0.484354913235, patience: 0
epoch [4/10000], loss:0.459526062012, patience: 0
epoch [5/10000], loss:0.443292319775, patience: 0
epoch [6/10000], loss:0.431527674198, patience: 0
epoch [7/10000], loss:0.421715199947, patience: 0
epoch [8/10000], loss:0.412439405918, patience: 0
epoch [9/10000], loss:0.40620726347, patience: 0
epoch [10/10000], loss:0.401567429304, patience: 0
epoch [11/10000], loss:0.395809143782, patience: 0
epoch [12/10000], loss:0.392387777567, patience: 0
epoch [13/10000], loss:0.389468938112, patience: 0
epoch [14/10000], loss:0.385871350765, patience: 0
epoch [15/10000], loss:0.381956726313, patience: 0
epoch [16/10000], loss:0.377608925104, patience: 0
epoch [17/10000], loss:0.379093021154, patience: 1
epoch [18/10000], loss:0.383710414171, patience: 2
epoch [19/10000], loss:0.377308160067, patience: 2
epoch [20/10000], loss:0.375165313482, pa

INFO:MDS:Starting Error: 45066359914.4
INFO:MDS:Epoch took: 10.2016329765
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (45066359914.4, 40688778593.9, 4377581320.51)
INFO:MDS:Epoch took: 10.4335730076
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (4377581320.51, 2249891587.1, 2127689733.41)
INFO:MDS:Epoch took: 10.4648411274
INFO:MDS:Turn 3: Radius 32: (prev, error decrease, error): (2127689733.41, 338689946.094, 1788999787.32)
INFO:MDS:Epoch took: 10.2979979515
INFO:MDS:Turn 4: Radius 32: (prev, error decrease, error): (1788999787.32, 61800420.5678, 1727199366.75)
INFO:MDS:Epoch took: 10.2159430981
INFO:MDS:Turn 5: Radius 32: (prev, error decrease, error): (1727199366.75, 126030515.251, 1601168851.5)
INFO:MDS:Epoch took: 10.1281728745
INFO:MDS:Turn 6: Radius 32: (prev, error decrease, error): (1601168851.5, -61307676.6913, 1662476528.19)
INFO:MDS:Epoch took: 10.3646559715
INFO:MDS:Turn 7: Radius 16.0: (prev, error decrease, error): (1662476528.19, 877636182.3

INFO:MDS:Epoch took: 10.3726899624
INFO:MDS:Turn 59: Radius 4.0: (prev, error decrease, error): (215574112.258, 428584.57868, 215145527.679)
INFO:MDS:Epoch took: 10.1742610931
INFO:MDS:Turn 60: Radius 4.0: (prev, error decrease, error): (215145527.679, 325961.809545, 214819565.87)
INFO:MDS:Epoch took: 10.3459589481
INFO:MDS:Turn 61: Radius 4.0: (prev, error decrease, error): (214819565.87, 302503.296634, 214517062.573)
INFO:MDS:Epoch took: 10.2154080868
INFO:MDS:Turn 62: Radius 4.0: (prev, error decrease, error): (214517062.573, 276148.021961, 214240914.551)
INFO:MDS:Epoch took: 10.4557211399
INFO:MDS:Turn 63: Radius 4.0: (prev, error decrease, error): (214240914.551, 255349.26636, 213985565.285)
INFO:MDS:Epoch took: 10.4708230495
INFO:MDS:Turn 64: Radius 4.0: (prev, error decrease, error): (213985565.285, 184262.889275, 213801302.396)
INFO:MDS:Epoch took: 10.3811340332
INFO:MDS:Turn 65: Radius 4.0: (prev, error decrease, error): (213801302.396, 163611.110273, 213637691.285)
INFO:MDS:E

...

INFO:MDS:Starting Error: 47297863179.4
INFO:MDS:Epoch took: 1.50148296356
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (47297863179.4, 38088834394.5, 9209028784.83)
INFO:MDS:Epoch took: 1.45647716522
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (9209028784.83, 3541633559.52, 5667395225.31)
INFO:MDS:Epoch took: 1.60336995125
INFO:MDS:Turn 3: Radius 32: (prev, error decrease, error): (5667395225.31, 462926361.575, 5204468863.74)
INFO:MDS:Epoch took: 1.45309019089
INFO:MDS:Turn 4: Radius 32: (prev, error decrease, error): (5204468863.74, 126660918.038, 5077807945.7)
INFO:MDS:Epoch took: 1.45621299744
INFO:MDS:Turn 5: Radius 32: (prev, error decrease, error): (5077807945.7, 100224502.47, 4977583443.23)
INFO:MDS:Epoch took: 1.45048999786
INFO:MDS:Turn 6: Radius 32: (prev, error decrease, error): (4977583443.23, -33948389.796, 5011531833.03)
INFO:MDS:Epoch took: 1.44632911682
INFO:MDS:Turn 7: Radius 16.0: (prev, error decrease, error): (5011531833.03, 1807155327.3

INFO:MDS:Epoch took: 1.45576095581
INFO:MDS:Turn 59: Radius 1.0: (prev, error decrease, error): (2416988181.11, 1252547.21206, 2415735633.9)
INFO:MDS:Epoch took: 1.52683520317
INFO:MDS:Turn 60: Radius 1.0: (prev, error decrease, error): (2415735633.9, 1056470.15511, 2414679163.75)
INFO:MDS:Epoch took: 1.66129302979
INFO:MDS:Turn 61: Radius 0.5: (prev, error decrease, error): (2414679163.75, 1478062.94749, 2413201100.8)
INFO:MDS:Epoch took: 1.65396213531
INFO:MDS:Turn 62: Radius 0.5: (prev, error decrease, error): (2413201100.8, 760397.922027, 2412440702.88)
INFO:MDS:Epoch took: 1.45190000534
INFO:MDS:Turn 63: Radius 0.25: (prev, error decrease, error): (2412440702.88, 600106.258731, 2411840596.62)
INFO:MDS:Epoch took: 1.4523499012
INFO:MDS:Turn 64: Radius 0.125: (prev, error decrease, error): (2411840596.62, 248416.256224, 2411592180.36)
INFO:MDS:Epoch took: 1.46192193031
INFO:MDS:Turn 65: Radius 0.0625: (prev, error decrease, error): (2411592180.36, 111193.072932, 2411480987.29)
INFO:

MDS.fit_transform took: 107.696325064 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: Modified LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: ISOMAP
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 3...
Checking Method: Autoencoder
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 2...
epoch [1/10000], loss:0.713360488415, patience: 0
epoch [2/10000], loss:0.66991341114, patience: 0
epoch [3/10000], loss:0.66225874424, patience: 0
epoch [4/10000], loss:0.658304870129, patience: 0
epoch [5/10000], loss:0.653691828251, patience: 0
epoch [6/10000], 

INFO:MDS:Starting Error: 47526146795.4
INFO:MDS:Epoch took: 1.06513094902
INFO:MDS:Turn 1: Radius 32: (prev, error decrease, error): (47526146795.4, 35136917647.8, 12389229147.5)
INFO:MDS:Epoch took: 1.05688500404
INFO:MDS:Turn 2: Radius 32: (prev, error decrease, error): (12389229147.5, 3248607441.74, 9140621705.78)
INFO:MDS:Epoch took: 1.06914019585
INFO:MDS:Turn 3: Radius 32: (prev, error decrease, error): (9140621705.78, 337271960.665, 8803349745.12)
INFO:MDS:Epoch took: 1.12206816673
INFO:MDS:Turn 4: Radius 32: (prev, error decrease, error): (8803349745.12, 434440152.241, 8368909592.88)
INFO:MDS:Epoch took: 1.29164195061
INFO:MDS:Turn 5: Radius 32: (prev, error decrease, error): (8368909592.88, -150866962.171, 8519776555.05)
INFO:MDS:Epoch took: 1.32201099396
INFO:MDS:Turn 6: Radius 16.0: (prev, error decrease, error): (8519776555.05, 3400180226.54, 5119596328.51)
INFO:MDS:Epoch took: 1.27715587616
INFO:MDS:Turn 7: Radius 16.0: (prev, error decrease, error): (5119596328.51, 140551

MDS.fit_transform took: 40.9249100685 sec
Checking Method: Truncated SVD
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: Spectral Embedding
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: Modified LLE
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: ISOMAP
Reducing Input from Dimension: 1582 to a Lower Embedded Manifold with dimensions: 2...
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Method: ISOMAP
Checking Method: LLE
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Met

In [17]:
from IPython.display import display
for target_dim in sorted(original_results.keys()):
    df = original_results[target_dim]
    print "Using Original Data"
    print display(df)

for target_dim in sorted(emobase_results1.keys()):
    df = emobase_results1[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    
for target_dim in sorted(emobase_results2.keys()):
    df = emobase_results2[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    
for target_dim in sorted(emobase_results3.keys()):
    df = emobase_results3[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)

Using Original Data


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,44.02,48.54,50.52,50.84,51.26,51.74,52.16,51.84,52.12,51.96
ISOMAP WA,42.82,47.06,49.06,49.50,49.78,50.16,50.68,50.46,50.58,50.52
LLE UA,44.64,47.72,51.20,51.30,52.54,52.72,53.04,53.40,53.58,53.22
LLE WA,44.00,47.26,50.64,51.00,52.10,52.40,52.70,53.04,52.96,52.74
Modified LLE UA,46.88,50.96,53.62,54.14,55.22,55.44,55.90,55.92,55.84,55.42
Modified LLE WA,46.52,50.30,52.80,53.48,54.42,54.58,54.96,55.20,55.14,54.74
Pattern Search MDS UA,43.02,46.78,48.20,48.66,49.50,49.24,49.52,49.74,50.22,49.90
Pattern Search MDS WA,41.94,45.22,46.56,47.32,48.04,47.70,47.86,47.94,48.34,48.02


None
For Target Dimension: 50


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,44.02,48.54,50.52,50.84,51.26,51.74,52.16,51.84,52.12,51.96
ISOMAP WA,42.82,47.06,49.06,49.50,49.78,50.16,50.68,50.46,50.58,50.52
LLE UA,44.64,47.72,51.20,51.30,52.54,52.72,53.04,53.40,53.58,53.22
LLE WA,44.00,47.26,50.64,51.00,52.10,52.40,52.70,53.04,52.96,52.74
Modified LLE UA,46.88,50.96,53.62,54.14,55.22,55.44,55.90,55.92,55.84,55.42
Modified LLE WA,46.52,50.30,52.80,53.48,54.42,54.58,54.96,55.20,55.14,54.74
Pattern Search MDS UA,43.02,46.78,48.20,48.66,49.50,49.24,49.52,49.74,50.22,49.90
Pattern Search MDS WA,41.94,45.22,46.56,47.32,48.04,47.70,47.86,47.94,48.34,48.02


None
For Target Dimension: 100


...

None
For Target Dimension: 10


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,44.34,46.36,48.14,49.06,49.82,49.76,50.52,50.34,50.34,50.56
ISOMAP WA,43.72,45.58,47.32,48.14,49.02,48.88,49.84,49.60,49.62,49.76
LLE UA,43.40,45.74,47.02,48.00,49.10,49.54,50.24,50.34,50.90,51.06
LLE WA,42.78,45.26,46.80,47.76,48.98,49.28,49.76,49.76,50.28,50.50
Modified LLE UA,44.52,47.70,49.68,50.42,51.10,51.54,51.70,52.28,52.24,52.42
Modified LLE WA,44.06,46.96,48.64,49.60,50.36,50.74,50.96,51.50,51.36,51.64
Pattern Search MDS UA,44.80,50.52,51.58,52.74,53.44,53.40,53.90,54.26,54.74,54.34
Pattern Search MDS WA,44.34,49.48,50.48,51.58,52.28,52.00,52.50,52.82,53.20,52.88


None
For Target Dimension: 25


,1,5,9,13,17,21,25,29,33,37
Autoencoder UA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Autoencoder WA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ISOMAP UA,44.84,47.66,49.10,50.24,50.52,50.80,50.98,51.34,51.68,51.60
ISOMAP WA,43.60,46.34,47.76,48.96,49.30,49.54,49.64,50.08,50.46,50.28
LLE UA,44.58,47.96,49.68,50.96,51.28,52.06,52.70,53.10,53.06,53.60
LLE WA,43.86,47.58,49.24,50.76,50.94,51.62,52.34,52.70,52.62,53.26
Modified LLE UA,44.98,48.04,50.74,51.44,52.50,53.08,53.40,53.76,53.94,53.60
Modified LLE WA,44.60,47.50,49.88,50.78,51.64,52.24,52.56,53.00,53.18,52.82
Pattern Search MDS UA,44.94,49.84,51.56,52.06,52.52,52.94,53.22,53.28,53.50,53.62
Pattern Search MDS WA,44.08,48.34,49.74,50.32,50.82,50.90,51.16,51.12,51.40,51.44


None


In [22]:
# create the experiment for EmoDB Speaker independent Experiments
def get_dataset_for_all_speakers(features_dic):
    speaker_indices = {}
    x_all_list = []
    Y_all = []
    prev_ind = 0
    for te_speaker, te_data in features_dic.items():  
        x_all_list.append(te_data['x'])
        Y_all += te_data['y']
        this_speaker_samples = len(te_data['y'])
        
        speaker_indices[te_speaker] = (prev_ind, prev_ind + this_speaker_samples)
        prev_ind += this_speaker_samples
        X_all = np.concatenate(x_all_list, axis=0)
    number_of_speakers = len(features_dic.keys())
    return X_all, Y_all, speaker_indices, number_of_speakers

def generate_speaker_independent_folds(X_all, Y_all, features_dic, speaker_indices):
    sorted_speakers = sorted(speaker_indices.keys())
    for (te_speaker, (st, et)) in speaker_indices.items():
        Y_te = Y_all[st:et]
        X_te = X_all[st:et, :]
        
        x_tr_list = []
        Y_tr = []
        for sp in sorted_speakers:
            if sp == te_speaker:
                continue
            st, et = speaker_indices[sp] 
            x_tr_list.append(X_all[st:et, :])
            Y_tr += Y_all[st:et]
        X_tr = np.concatenate(x_tr_list, axis=0)    
        
        yield te_speaker, X_te, Y_te, X_tr, Y_tr 


        
def run_DR(target_dims, methods_to_test, data_dic, saveto=None):
    save_file = os.path.join('../', 'cache', saveto)
    if saveto is not None and os.path.isfile(save_file):
        with open(save_file, 'rb') as fd:
            reduced = pickle.load(fd)
        return reduced
    X_all, Y_all, speaker_indices, number_of_sessions = get_dataset_for_all_speakers(data_dic)
    # normalize the input vectors 
    X_high = StandardScaler().fit_transform(X_all)
    
    print(X_high.shape)
    
    reduced = {}
    original_dim = X_all.shape[1]
    for target_dim in target_dims:
        reduced[target_dim] = {}
        manifold_methods = get_manifold_methods(original_dim, target_dim)
    #     methods_to_test = manifold_methods.keys()
        for selected_method in methods_to_test:            
            print('Checking Method: {}'.format(selected_method))
            
            print('Reducing Input from Dimension: {} to a Lower Embedded Manifold with dimensions: {}...'.format(
                   X_high.shape[1], target_dim))
            try:
                obj = manifold_methods[selected_method]['object']
                if selected_method == 'Pattern Search MDS':
                    d_goal = multidimensional.common.DISTANCE_MATRIX(X_high.astype(np.float64))
#                    d_goal = 1.0 - np.corrcoef(X_high.astype(np.float64))
#                    np.fill_diagonal(d_goal, 0)
                    X_low = obj.fit_transform(d_goal)
                else:
                    X_low = obj.fit_transform(X_high)
            except Exception as e:
                print(e)
                X_low = None
            reduced[target_dim][selected_method] = X_low
    if saveto is not None:
        with open(save_file, 'wb') as fd:
            pickle.dump(reduced, fd)
    return reduced

def run_speaker_independent_KNN(n_neighbors, reduced, data_dic):

    X_all, Y_all, speaker_indices, number_of_speakers = get_dataset_for_all_speakers(data_dic)
    df_results = {}
    # normalize the input vectors 
    X_high = StandardScaler().fit_transform(X_all)
    
    print X_high.shape 

    for target_dim, methods in reduced.iteritems():
        methods_metrics = {}
        for selected_method, X_low in methods.iteritems():
            print('Checking Method: {}'.format(selected_method))
            metrics_l = {'uw_acc': dict([(k, 0.0) for k in n_neighbors]), 'w_acc': dict([(k, 0.0) for k in n_neighbors])}
            if X_low is None:
                methods_metrics[selected_method+' UA'] = metrics_l['uw_acc']
                methods_metrics[selected_method+' WA'] = metrics_l['w_acc']
                continue             
            
            for k in n_neighbors:
                knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', algorithm='brute', leaf_size=30, 
                                           p=2, metric='minkowski', metric_params=None, n_jobs=8)

                speaker_folds = generate_speaker_independent_folds(X_low, Y_all, data_dic, speaker_indices)
                for te_speaker, X_te, Y_te, X_tr, Y_tr in speaker_folds:                    
                    try:
                        knn.fit(X_tr, Y_tr) 
                        Y_predicted = knn.predict(X_te)

                        w_acc = accuracy_score(Y_predicted, Y_te)
                        cmat = confusion_matrix(Y_te, Y_predicted)
                        with np.errstate(divide='ignore'):
                            uw_acc = (cmat.diagonal() / (1.0 * cmat.sum(axis=1) + 1e-6)).mean()
                            if np.isnan(uw_acc):
                                uw_acc = 0.
                        w_acc = round(w_acc*100,0)
                        uw_acc = round(uw_acc*100,)
                        metrics_l['uw_acc'][k] += uw_acc/number_of_speakers
                        metrics_l['w_acc'][k] += w_acc/number_of_speakers
                    except:
                        metrics_l['uw_acc'][k] += 0.
                        metrics_l['w_acc'][k] += 0.
                    
            methods_metrics[selected_method+' UA'] = metrics_l['uw_acc']
            methods_metrics[selected_method+' WA'] = metrics_l['w_acc']

        df = pd.DataFrame.from_dict(methods_metrics, orient="index")
        df_results[target_dim] = df[sorted(df.columns)]
        
    return df_results

In [23]:
# Define parameters for Speaker Independent Experiments 
n_neighbors = np.arange(1, 40, 4)
target_dims = [2, 5, 10, 25]

# Find all appropriate files 
data_path = '/home/george/all_BERLIN_features/'
berlin_l_feats_p = data_path + 'linear/BERLIN_linear_emobase2010'
# nl_feats_l = glob.glob( IEMOCAP_data_path + '/utterance/*.dat')
# nl_feats_p = nl_feats_l.pop()
berlin_nl_feats_p = os.path.join(data_path, 
             'rqa/utterance/BERLIN-rqa-ad_hoc-tau-7-manhattan-recurrence_rate-0.15-dur-0.02-fs-16000.dat')


In [ ]:
methods_to_test = ['Autoencoder', 'Pattern Search MDS', 'MDS SMACOF','Truncated SVD', 'Spectral Embedding', 'LLE', 
                   'Modified LLE', 'ISOMAP']   
# methods_to_test = ['Pattern Search MDS']
# methods_to_test = ['Truncated SVD']

target_dims = [100, 50, 25, 10, 5, 3, 2]
# methods_to_test = ['Truncated SVD', 'Spectral Embedding']
data_dic = IEMOCAP_loader.get_fused_features([berlin_nl_feats_p])
berlin_original_nl_results = run_speaker_independent_KNN(n_neighbors, reduced, data_dic)
reduced = run_DR(target_dims, methods_to_test, data_dic, saveto='dr-rqa-berlin.p')
berlin_nl_results = run_speaker_independent_KNN(n_neighbors, reduced, data_dic)

with open('dr-rqa-berlin-results.p', 'wb') as fd:
    pickle.dump(berlin_nl_results, fd)
    
with open('original-rqa-berlin-results.p', 'wb') as fd:
    pickle.dump(berlin_original_nl_results, fd)


(535, 432)
Checking Method: Pattern Search MDS
Checking Method: Modified LLE
Checking Method: Spectral Embedding
Checking Method: Autoencoder
Checking Method: Truncated SVD
Checking Method: ISOMAP


In [ ]:
data_dic = IEMOCAP_loader.get_fused_features([berlin_l_feats_p])
berlin_original_l_results = run_speaker_independent_KNN(n_neighbors, reduced, data_dic)
reduced = run_DR(target_dims, methods_to_test, data_dic, saveto='dr-emobase-berlin.p')
berlin_l_results = run_speaker_independent_KNN(n_neighbors, reduced, data_dic)

with open('dr-emobase-berlin-results.p', 'wb') as fd:
    pickle.dump(berlin_nl_results, fd)
    
with open('original-emobase-berlin-results.p', 'wb') as fd:
    pickle.dump(berlin_original_nl_results, fd)


In [ ]:
data_dic = IEMOCAP_loader.get_fused_features([berlin_l_feats_p, berlin_nl_feats_p])
berlin_original_fused_results = run_speaker_independent_KNN(n_neighbors, reduced, data_dic)
reduced = run_DR(target_dims, methods_to_test, data_dic, saveto='dr-fused-berlin.p')
berlin_fused_results = run_speaker_independent_KNN(n_neighbors, reduced, data_dic)

with open('dr-fused-berlin-results.p', 'wb') as fd:
    pickle.dump(berlin_nl_results, fd)
    
with open('original-fused-berlin-results.p', 'wb') as fd:
    pickle.dump(berlin_original_nl_results, fd)


In [ ]:
from IPython.display import display
print "Using RQA Feature Set and Dimensionality Reduction..."

def latex_preformat_print(df):
    methods = {}
    for ind in df.index.values:
        if not ind[:-3] in methods and ind[-2:] == 'WA':
            methods[ind[:-3]] = list(df[[1,5,9,13,17,21]].loc[ind])
    for ind in df.index.values:
        if ind[-2:] == 'UA':
            methods[ind[:-3]] += list(df[[1,5,9,13,17,21]].loc[ind])
    
    df = pd.DataFrame.from_dict(methods, orient="index")
    print df.to_latex()

for target_dim in sorted(berlin_original_nl_results.keys()):
    df = berlin_original_nl_results[target_dim]
    print "Using Original Data"
    print display(df)
    latex_preformat_print(df)
    
for target_dim in sorted(berlin_nl_results.keys()):
    df = berlin_nl_results[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    latex_preformat_print(df)

In [ ]:
from IPython.display import display
print "Using Emobase Features and Dimensionality Reduction..."

for target_dim in sorted(berlin_original_l_results.keys()):
    df = berlin_original_l_results[target_dim]
    print "Using Original Data"
    print display(df)
    latex_preformat_print(df)

for target_dim in sorted(berlin_l_results.keys()):
    df = berlin_l_results[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    latex_preformat_print(df)

In [ ]:
from IPython.display import display
print "Using Fused Features and Dimensionality Reduction..."

for target_dim in sorted(berlin_original_fused_results.keys()):
    df = berlin_original_fused_results[target_dim]
    print "Using Original Data"
    print display(df)
    latex_preformat_print(df)

for target_dim in sorted(berlin_fused_results.keys()):
    df = berlin_fused_results[target_dim]
    print "For Target Dimension: {}".format(target_dim)
    print display(df)
    latex_preformat_print(df)